# 🌾 DenseNet201-SE — Paddy Disease Classification

Implementasi **DenseNet201 + Squeeze & Excitation (SE) Block** untuk klasifikasi penyakit daun padi.

## ✨ Keunggulan Notebook Ini

| Fitur | Detail |
|---|---|
| **Arsitektur** | DenseNet201 + SE Block + GAP + Dropout + Softmax |
| **Segmentasi** | **GrabCut Auto-Seed** (lebih robust dari HSV) |
| **Training** | GrabCut diterapkan di training & inference — **no domain shift** |
| **Loss** | Focal Loss (γ=2.0, α=0.25) — tahan class imbalance |
| **Training** | 2-Stage: Freeze Head → Fine-Tuning Backbone |
| **Compatibility** | Keras 3 (`@register_keras_serializable`) |

## 📋 Urutan Menjalankan

### Training (dari awal):
```
Cell 1  → Import library
Cell 2  → Konfigurasi (set USE_GRABCUT_TRAINING)
Cell 2b → GrabCut utility functions ← WAJIB
Cell 3  → Dataset loading
Cell 4  → Train/Val split
Cell 5  → tf.data pipeline (+ GrabCut opsional)
Cell 6  → Custom components (FocalLoss, DenseNetPreprocess)
Cell 7  → Build model
Cell 8  → Class weights
Cell 9  → Plot helper
Cell 10 → Stage 1: Train head
Cell 11 → Stage 2: Fine-tuning
Cell 12 → Evaluasi
Cell 13 → Info model
```

### Inference saja (model sudah ada):
```
Cell 2b → GrabCut functions ← WAJIB
Cell 13a → Load model
Cell 14b → Prediksi 1 gambar
Cell 14c → Batch prediksi → CSV
```


In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import tensorflow as tf
import keras
from keras import layers

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)

In [ ]:
# ----------------------------
# 1) Configuration
# ----------------------------
DATASET_DIR   = "paddy-disease-classification"
TRAIN_IMG_DIR = os.path.join(DATASET_DIR, "train_images")
TEST_IMG_DIR  = os.path.join(DATASET_DIR, "test_images")
SAMPLE_SUB    = os.path.join(DATASET_DIR, "sample_submission.csv")

OUTPUT_DIR = "outputs_densenet201_se_standalone"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

IMG_SIZE   = (224, 224)
BATCH_SIZE = 16
AUTOTUNE   = tf.data.AUTOTUNE

EPOCHS_STAGE1 = 10
EPOCHS_STAGE2 = 15

LR1           = 1e-3
LR2           = 1e-5
UNFREEZE_LAST = 30

DROPOUT  = 0.4
SE_RATIO = 16

USE_FOCAL_LOSS = True
GAMMA = 2.0
ALPHA = 0.25
# GrabCut di training: True=konsisten inference/training, False=RAW
USE_GRABCUT_TRAINING = True


In [ ]:
# ----------------------------
# 2b) GrabCut Auto-Seed Segmentation — Didefinisikan Sekali, Dipakai Training & Inference
# ----------------------------
try:
    import cv2
    print(f"[INFO] ✅ OpenCV {cv2.__version__}")
except ImportError:
    raise ImportError("Install dulu: pip install opencv-python")
import numpy as np

# ── 1. UTILITAS ────────────────────────────────────────────────────────────

def keep_largest_component(mask_u8):
    """Ambil komponen putih terbesar."""
    mask = (mask_u8 > 0).astype(np.uint8)
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num <= 1: return (mask * 255).astype(np.uint8)
    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    return (labels == largest).astype(np.uint8) * 255

def refine_mask(mask_u8, close_k=15, open_k=7, close_it=2, open_it=1):
    """CLOSE (tutup lubang) → OPEN (hapus noise kecil)."""
    m  = (mask_u8 > 0).astype(np.uint8) * 255
    kc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_k, close_k))
    ko = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_k,  open_k))
    m  = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kc, iterations=close_it)
    m  = cv2.morphologyEx(m, cv2.MORPH_OPEN,  ko, iterations=open_it)
    return m

def crop_by_mask(bgr, mask_u8, pad=18):
    """Crop pixel-accurate ke bounding box mask."""
    ys, xs = np.where(mask_u8 > 0)
    if len(xs) == 0: return bgr, (0, 0, bgr.shape[1], bgr.shape[0])
    h, w = bgr.shape[:2]
    x1 = max(0, xs.min()-pad); y1 = max(0, ys.min()-pad)
    x2 = min(w-1, xs.max()+pad); y2 = min(h-1, ys.max()+pad)
    return bgr[y1:y2+1, x1:x2+1], (x1, y1, x2, y2)

# ── 2. GRABCUT AUTO-SEED ───────────────────────────────────────────────────

def grabcut_leaf_mask_autoseed(bgr, iters=5):
    """
    GrabCut dengan 3-zona seed otomatis:
      Sure FG  → H=[25-95], S>=60, V>=40  (hijau yekin = daun)
      Sure BG  → V<=20 | V>=245 | S<=10   (gelap/terang/abu = background)
      Unknown  → sisa piksel, GrabCut putuskan via GMM
    Fallback   → rect-based jika FG seed < 0.5% gambar.
    """
    h, w  = bgr.shape[:2]
    hsv   = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    H_ch, S_ch, V_ch = cv2.split(hsv)

    sure_fg = ((H_ch>=25)&(H_ch<=95)&(S_ch>=60)&(V_ch>=40)).astype(np.uint8)*255
    sure_bg = ((V_ch<=20)|(V_ch>=245)|(S_ch<=10)).astype(np.uint8)*255

    k7 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    sure_fg = cv2.morphologyEx(sure_fg, cv2.MORPH_OPEN,  k7, iterations=1)
    sure_fg = cv2.morphologyEx(sure_fg, cv2.MORPH_CLOSE, k7, iterations=2)
    sure_bg = cv2.morphologyEx(sure_bg, cv2.MORPH_OPEN,  k7, iterations=1)

    bgdM = np.zeros((1,65), np.float64)
    fgdM = np.zeros((1,65), np.float64)

    if cv2.countNonZero(sure_fg) < 0.005*(h*w):
        # Fallback: rect tengah
        gc = np.zeros((h,w), np.uint8)
        cv2.grabCut(bgr, gc, (10,10,w-20,h-20), bgdM, fgdM, iters, cv2.GC_INIT_WITH_RECT)
    else:
        gc = np.full((h,w), cv2.GC_PR_BGD, np.uint8)
        gc[sure_bg>0] = cv2.GC_BGD
        gc[sure_fg>0] = cv2.GC_FGD
        cv2.grabCut(bgr, gc, None, bgdM, fgdM, iters, cv2.GC_INIT_WITH_MASK)

    out = np.where((gc==cv2.GC_FGD)|(gc==cv2.GC_PR_FGD), 255, 0).astype(np.uint8)
    out = refine_mask(out, close_k=21, open_k=9, close_it=2, open_it=1)
    out = keep_largest_component(out)
    return out

# ── 3. PIPELINE LENGKAP ────────────────────────────────────────────────────

def preprocess_leaf_grabcut(bgr, target_size=(224,224), pad=18):
    """
    GrabCut → BG hitam ⬛ → crop ROI → resize → RGB float32.
    Dipakai di: training pipeline + inference.
    """
    leaf_mask  = grabcut_leaf_mask_autoseed(bgr, iters=5)
    leaf_only  = cv2.bitwise_and(bgr, bgr, mask=leaf_mask)
    crop, bbox = crop_by_mask(leaf_only, leaf_mask, pad=pad)
    crop_rsz   = cv2.resize(crop, target_size, interpolation=cv2.INTER_AREA)
    rgb_model  = cv2.cvtColor(crop_rsz, cv2.COLOR_BGR2RGB).astype(np.float32)
    return rgb_model, leaf_mask, leaf_only, bbox

# ── 4. TF.NUMPY_FUNCTION WRAPPER ──────────────────────────────────────────

def _tf_grabcut_map(image, label):
    """
    Wrapper tf.numpy_function untuk integrasi ke tf.data.
    Dipanggil lewat .map() saat training/validasi.
    Input : image tensor (H,W,3) float32 0..255 (RGB dari decode_image)
    Output: image tensor (224,224,3) float32 — sudah tersegmentasi GrabCut
    """
    def _np_fn(img_np):
        img_np = np.clip(img_np, 0, 255).astype(np.uint8)
        bgr    = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        rgb, _, _, _ = preprocess_leaf_grabcut(bgr, target_size=IMG_SIZE)
        return rgb.astype(np.float32)
    out = tf.numpy_function(_np_fn, [image], Tout=tf.float32)
    out.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
    return out, label

print("✅ GrabCut Auto-Seed functions siap.")
print(f"   USE_GRABCUT_TRAINING = {USE_GRABCUT_TRAINING}")
print("   True  → training + inference pakai GrabCut (NO domain shift)")
print("   False → training RAW, inference GrabCut (ada domain shift)")


In [ ]:
# ----------------------------
# 2) Dynamic Dataset Loading (Folder Scan)
# ----------------------------
filepaths = []
labels    = []

classes = sorted(os.listdir(TRAIN_IMG_DIR))
classes = [c for c in classes if os.path.isdir(os.path.join(TRAIN_IMG_DIR, c))]
print(f"[INFO] Classes found ({len(classes)}): {classes}")

class_counts = {}
for label in classes:
    class_dir = os.path.join(TRAIN_IMG_DIR, label)
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    class_counts[label] = len(images)
    for img in images:
        filepaths.append(os.path.join(class_dir, img))
        labels.append(label)

df = pd.DataFrame({'filepath': filepaths, 'label': labels})

print(f"\n[INFO] Total Dataset: {len(df)} images")
print("Distribution per class:")
print(pd.Series(class_counts))

class_names  = sorted(df['label'].unique().tolist())
num_classes  = len(class_names)
class_to_idx = {c: i for i, c in enumerate(class_names)}

with open(os.path.join(OUTPUT_DIR, "class_names.json"), "w") as f:
    json.dump(class_names, f, indent=2)

In [ ]:
# ----------------------------
# 3) Train/Val Split (Stratified 90:10)
# ----------------------------
train_df, val_df = train_test_split(
    df, test_size=0.10, random_state=SEED, stratify=df["label"]
)
train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

print(f"Training Set   : {len(train_df)} images")
print(f"Validation Set : {len(val_df)} images")

In [ ]:
# ----------------------------
# 4) tf.data Input Pipeline (opsional GrabCut preprocessing)
# ----------------------------
def decode_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE, method="bilinear")
    return tf.cast(img, tf.float32)

def process_path(path, label_idx):
    return decode_image(path), tf.one_hot(label_idx, num_classes)

def make_ds(dataframe, training=True):
    paths  = dataframe["filepath"].values
    idxs   = dataframe["label"].map(class_to_idx).values.astype('int32')
    ds     = tf.data.Dataset.from_tensor_slices((paths, idxs))
    if training:
        ds = ds.shuffle(buffer_size=min(len(dataframe), 5000), seed=SEED)
    ds = ds.map(process_path, num_parallel_calls=AUTOTUNE)
    if USE_GRABCUT_TRAINING:
        # Terapkan GrabCut ke setiap gambar — konsisten dengan inference
        ds = ds.map(_tf_grabcut_map, num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = make_ds(train_df, training=True)
val_ds   = make_ds(val_df,   training=False)

mode = 'GrabCut Auto-Seed' if USE_GRABCUT_TRAINING else 'RAW (tanpa segmentasi)'
print(f"[INFO] Training pipeline : {mode}")
print(f"[INFO] Train batches     : {len(train_ds)}")
print(f"[INFO] Val   batches     : {len(val_ds)}")


In [ ]:
# ----------------------------
# 5) Keras 3 Serializable Components
# ----------------------------
@keras.saving.register_keras_serializable(package="custom")
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, from_logits=False, name="focal_loss"):
        super().__init__(name=name)
        self.gamma = gamma
        self.alpha = alpha
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        if self.from_logits:
            y_pred = tf.nn.softmax(y_pred)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        ce     = -y_true * tf.math.log(y_pred)
        weight = self.alpha * tf.pow(1.0 - y_pred, self.gamma)
        return tf.reduce_sum(weight * ce, axis=-1)

    def get_config(self):
        return {"gamma": self.gamma, "alpha": self.alpha, "from_logits": self.from_logits, "name": self.name}

@keras.saving.register_keras_serializable(package="custom")
class DenseNetPreprocess(tf.keras.layers.Layer):
    def call(self, x):
        return tf.keras.applications.densenet.preprocess_input(x)
    def get_config(self):
        return {}

LOSS_FN = FocalLoss(gamma=GAMMA, alpha=ALPHA) if USE_FOCAL_LOSS else "categorical_crossentropy"
print("Loss Function:", LOSS_FN.name if hasattr(LOSS_FN, 'name') else LOSS_FN)

In [ ]:
# ----------------------------
# 6) Model Architecture (DenseNet201 + SE Block)
# ----------------------------
def se_block(x, ratio=16, name="se"):
    c  = int(x.shape[-1])
    se = tf.keras.layers.GlobalAveragePooling2D(name=f"{name}_gap")(x)
    se = tf.keras.layers.Dense(max(1, c // ratio), activation="relu",    name=f"{name}_fc1")(se)
    se = tf.keras.layers.Dense(c,                  activation="sigmoid", name=f"{name}_fc2")(se)
    se = tf.keras.layers.Reshape((1, 1, c),                              name=f"{name}_reshape")(se)
    return tf.keras.layers.Multiply(name=f"{name}_scale")([x, se])

def build_densenet201_se(num_classes, dropout=DROPOUT, se_ratio=SE_RATIO, name="DenseNet201_SE"):
    backbone = tf.keras.applications.DenseNet201(
        include_top=False, weights="imagenet",
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )
    backbone.trainable = False

    inp = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    x   = DenseNetPreprocess(name="densenet_preprocess")(inp)
    x   = backbone(x, training=False)
    x   = se_block(x, ratio=se_ratio, name="se_block")
    x   = tf.keras.layers.GlobalAveragePooling2D()(x)
    x   = tf.keras.layers.Dropout(dropout)(x)
    out = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    return tf.keras.Model(inp, out, name=name)

model = build_densenet201_se(num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(LR1), loss=LOSS_FN, metrics=["accuracy"])
model.summary()

In [ ]:
# ----------------------------
# 7) Class Weights
# ----------------------------
classes_idx  = train_df["label"].map(class_to_idx).values
cw           = compute_class_weight(class_weight="balanced", classes=np.unique(classes_idx), y=classes_idx)
class_weight = {i: float(w) for i, w in enumerate(cw)}
print("Class Weights:", class_weight)

In [ ]:
# ----------------------------
# 8) Train & Eval Plot Helper
# ----------------------------
def plot_history(history, stage_name="Training"):
    acc      = history.history['accuracy']
    val_acc  = history.history['val_accuracy']
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    epochs   = range(1, len(acc) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))
    fig.suptitle(f"Train Evaluation — {stage_name}", fontsize=14, fontweight='bold')

    ax1.plot(epochs, acc,     'bo-', label='Train Accuracy')
    ax1.plot(epochs, val_acc, 'ro-', label='Val Accuracy')
    ax1.set_title('Accuracy'); ax1.set_xlabel('Epoch'); ax1.set_ylabel('Accuracy')
    ax1.legend(); ax1.grid(True, alpha=0.3)

    ax2.plot(epochs, loss,     'bo-', label='Train Loss')
    ax2.plot(epochs, val_loss, 'ro-', label='Val Loss')
    ax2.set_title('Loss'); ax2.set_xlabel('Epoch'); ax2.set_ylabel('Loss')
    ax2.legend(); ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# ----------------------------
# 9) STAGE 1: Train Head Only
# ----------------------------
checkpoint_path = os.path.join(OUTPUT_DIR, "best_stage1.keras")

callbacks_stage1 = [
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
]

print("\n🚀 Starting Stage 1 Training (Head Only)...")
history1 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_STAGE1, callbacks=callbacks_stage1, class_weight=class_weight
)
plot_history(history1, "Stage 1 — Head Training")

In [ ]:
# ----------------------------
# 10) STAGE 2: Fine Tuning
# ----------------------------
model = tf.keras.models.load_model(
    checkpoint_path,
    custom_objects={"FocalLoss": FocalLoss, "DenseNetPreprocess": DenseNetPreprocess}
)

backbone = model.layers[2]
backbone.trainable = True

for layer in backbone.layers[:-UNFREEZE_LAST]:
    layer.trainable = False
for layer in backbone.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(LR2), loss=LOSS_FN, metrics=["accuracy"])

final_checkpoint_path = os.path.join(OUTPUT_DIR, "densenet201_se_final.keras")

callbacks_stage2 = [
    tf.keras.callbacks.ModelCheckpoint(final_checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
]

print(f"\n🚀 Starting Stage 2 Fine-Tuning (Unfreezing last {UNFREEZE_LAST} layers)...")
history2 = model.fit(
    train_ds, validation_data=val_ds,
    epochs=EPOCHS_STAGE2, callbacks=callbacks_stage2, class_weight=class_weight
)
plot_history(history2, "Stage 2 — Fine Tuning")

In [ ]:
# ----------------------------
# 11) Evaluasi: Confusion Matrix & Report
# ----------------------------
best_model = tf.keras.models.load_model(
    final_checkpoint_path,
    custom_objects={"FocalLoss": FocalLoss, "DenseNetPreprocess": DenseNetPreprocess}
)

print("\n📊 Generating Evaluation Metrics...")

y_pred, y_true = [], []
for bx, by in val_ds:
    probs = best_model.predict(bx, verbose=0)
    y_pred.extend(np.argmax(probs, axis=1))
    y_true.extend(np.argmax(by.numpy(), axis=1))

print("\n✅ Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(11, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted'); plt.ylabel('Ground Truth')
plt.title('Confusion Matrix — Validation Set')
plt.tight_layout()
plt.show()

In [ ]:
# ----------------------------
# 12) Save Model
# ----------------------------
print("\n💾 Model saved successfully at:")
print(f"   - {final_checkpoint_path}")
print("   (Gunakan file ini untuk inference atau deployment)")

---
## 🔍 13) Inference Helper

Gunakan bagian ini untuk **menguji model** pada gambar baru.
**Tidak perlu training ulang** — cukup load model yang sudah tersimpan.

> ⚠️ **Cell 2b (GrabCut functions) WAJIB dijalankan dulu** sebelum Cell 13a.

| Cell | Fungsi |
|:---:|---|
| **13a** | Load library + model + class names |
| **13b** | Prediksi 1 gambar (tanpa segmentasi — baseline) |
| **13c** | Batch prediksi folder `test_images/` → CSV (tanpa segmentasi) |

Untuk prediksi **dengan GrabCut segmentation**, gunakan **Section 14** (Cell 14b / 14c).


In [ ]:
# ----------------------------
# 13a) Load Model + Library untuk Inference
# WAJIB dijalankan pertama sebelum Cell 13b / 13c / 14b / 14c
# ----------------------------
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import tensorflow as tf
import keras

# ============================================================
# AUTO-DETECT path model — bekerja di CWD manapun
# ============================================================
def _find_model_path():
    """Cari file model .keras secara otomatis dari beberapa lokasi kandidat."""
    # Folder project (absolut, diambil dari lokasi notebook ini)
    base_candidates = [
        os.getcwd(),
        r"c:\Users\azzik\Documents\Tugas Akhir\Skripsi\Model DenseNet-201",
    ]
    model_subpaths = [
        os.path.join("model", "outputs_densenet201_se_standalone", "densenet201_se_final.keras"),
        os.path.join("outputs_densenet201_se_standalone", "densenet201_se_final.keras"),
    ]
    for base in base_candidates:
        for sub in model_subpaths:
            candidate = os.path.join(base, sub)
            if os.path.exists(candidate):
                return candidate
    return None

def _find_class_names_path():
    """Cari file class_names.json secara otomatis."""
    base_candidates = [
        os.getcwd(),
        r"c:\Users\azzik\Documents\Tugas Akhir\Skripsi\Model DenseNet-201",
    ]
    cn_subpaths = [
        os.path.join("model", "outputs_densenet201_se_standalone", "class_names.json"),
        os.path.join("outputs_densenet201_se_standalone", "class_names.json"),
        "class_names.json",
    ]
    for base in base_candidates:
        for sub in cn_subpaths:
            candidate = os.path.join(base, sub)
            if os.path.exists(candidate):
                return candidate
    return None

# Resolusi path
print(f"[INFO] Working Directory : {os.getcwd()}")

MODEL_PATH       = _find_model_path()
CLASS_NAMES_PATH = _find_class_names_path()
CUSTOM_TEST_DIR  = "test_images"   # Folder gambar uji (di LUAR paddy-disease-classification)
IMG_SIZE         = (224, 224)

if MODEL_PATH is None:
    raise FileNotFoundError(
        "❌ File model tidak ditemukan!\n"
        "   Pastikan file 'densenet201_se_final.keras' ada di:\n"
        "   model/outputs_densenet201_se_standalone/densenet201_se_final.keras"
    )
if CLASS_NAMES_PATH is None:
    raise FileNotFoundError(
        "❌ File class_names.json tidak ditemukan!\n"
        "   Pastikan file ada di folder output yang sama dengan model."
    )

print(f"[INFO] Model ditemukan  : {MODEL_PATH}")
print(f"[INFO] Classes path     : {CLASS_NAMES_PATH}")

# Re-register custom components
@keras.saving.register_keras_serializable(package="custom")
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, from_logits=False, name="focal_loss"):
        super().__init__(name=name)
        self.gamma = gamma; self.alpha = alpha; self.from_logits = from_logits
    def call(self, y_true, y_pred):
        if self.from_logits: y_pred = tf.nn.softmax(y_pred)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        ce = -tf.cast(y_true, tf.float32) * tf.math.log(y_pred)
        return tf.reduce_sum(self.alpha * tf.pow(1.0 - y_pred, self.gamma) * ce, axis=-1)
    def get_config(self):
        return {"gamma": self.gamma, "alpha": self.alpha, "from_logits": self.from_logits, "name": self.name}

@keras.saving.register_keras_serializable(package="custom")
class DenseNetPreprocess(tf.keras.layers.Layer):
    def call(self, x): return tf.keras.applications.densenet.preprocess_input(x)
    def get_config(self): return {}

CUSTOM_OBJ = {"FocalLoss": FocalLoss, "DenseNetPreprocess": DenseNetPreprocess}

# Load model
print(f"\n[INFO] Loading model...")
inf_model = tf.keras.models.load_model(MODEL_PATH, custom_objects=CUSTOM_OBJ)
print("[INFO] ✅ Model berhasil di-load!")

# Load class names
with open(CLASS_NAMES_PATH, "r") as f:
    class_names_inf = json.load(f)

print(f"[INFO] Classes ({len(class_names_inf)}): {class_names_inf}")
print(f"[INFO] Test Images Dir  : '{CUSTOM_TEST_DIR}'")
print("\n✅ Siap! Jalankan Cell 13b, 13c, 14b, atau 14c.")

### 13b. Prediksi 1 Gambar — Tanpa Segmentasi (Baseline)

Digunakan untuk **membandingkan** hasil dengan/tanpa GrabCut segmentation.
Ubah `IMAGE_FILE` ke nama gambar di folder `test_images/`.


In [ ]:
# ----------------------------
# 13b) Prediksi SATU Gambar (Tanpa Segmentasi)
# ----------------------------
def predict_single_image(model, class_names, image_path, img_size=(224, 224)):
    import matplotlib.patches as mpatches
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"❌ Gambar tidak ditemukan: '{image_path}'")

    img_raw     = tf.io.read_file(image_path)
    img         = tf.image.decode_image(img_raw, channels=3, expand_animations=False)
    img_display = img.numpy().astype('uint8')
    img         = tf.image.resize(img, img_size, method="bilinear")
    img         = tf.cast(img, tf.float32)
    img         = tf.expand_dims(img, axis=0)

    probs      = model.predict(img, verbose=0)[0]
    pred_idx   = int(np.argmax(probs))
    pred_label = class_names[pred_idx]
    confidence = float(probs[pred_idx])

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle("Prediksi DenseNet201-SE (Tanpa Segmentasi)", fontsize=13, fontweight='bold')
    ax1.imshow(img_display); ax1.axis('off')
    color = '#27ae60' if confidence >= 0.80 else ('#e67e22' if confidence >= 0.50 else '#e74c3c')
    ax1.set_title(f"🏷️  {pred_label.replace('_',' ').title()}\nConfidence: {confidence*100:.2f}%",
                  fontsize=12, fontweight='bold', color=color)
    colors = ['#27ae60' if i == pred_idx else '#3498db' for i in range(len(class_names))]
    bars   = ax2.barh(class_names, probs * 100, color=colors, edgecolor='white', height=0.6)
    ax2.set_xlabel('Probability (%)'); ax2.set_title('Distribusi Probabilitas')
    ax2.set_xlim(0, 105)
    for bar, val in zip(bars, probs):
        ax2.text(val * 100 + 0.8, bar.get_y() + bar.get_height() / 2,
                 f'{val*100:.1f}%', va='center', fontsize=9)
    ax2.legend(handles=[mpatches.Patch(color='#27ae60', label='Predicted'),
                        mpatches.Patch(color='#3498db', label='Others')], loc='lower right')
    ax2.grid(axis='x', alpha=0.3)
    plt.tight_layout(); plt.show()

    print(f"\n🎯 Kelas: {pred_label} | Confidence: {confidence*100:.2f}%")
    return pred_label, confidence, probs

IMAGE_FILE = "image.png"   # ← Ganti nama file
IMAGE_PATH = os.path.join(CUSTOM_TEST_DIR, IMAGE_FILE)
pred_label, confidence, all_probs = predict_single_image(inf_model, class_names_inf, IMAGE_PATH)

### 13c. Batch Prediksi `test_images/` → CSV — Tanpa Segmentasi (Baseline)

Hasilnya disimpan ke `test_images/prediction_results.csv`.
Bandingkan dengan `prediction_results_grabcut.csv` dari Cell 14c.


In [ ]:
# ----------------------------
# 13c) Batch Prediction → CSV (Tanpa Segmentasi)
# ----------------------------
def predict_all_in_folder(model, class_names, test_dir, img_size=(224, 224), batch_size=16):
    valid_ext = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
    all_files = sorted([f for f in os.listdir(test_dir) if f.endswith(valid_ext)])
    if not all_files:
        print(f"[WARN] Tidak ada gambar di '{test_dir}'."); return None
    print(f"[INFO] Ditemukan {len(all_files)} gambar")
    test_paths = [os.path.join(test_dir, f) for f in all_files]

    def load_img(path):
        raw = tf.io.read_file(path)
        img = tf.image.decode_image(raw, channels=3, expand_animations=False)
        img = tf.image.resize(img, img_size, method="bilinear")
        return tf.cast(img, tf.float32)

    test_ds = (tf.data.Dataset.from_tensor_slices(test_paths)
               .map(load_img, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(batch_size).prefetch(tf.data.AUTOTUNE))

    print("[INFO] Running inference...")
    all_probs   = np.concatenate([model.predict(b, verbose=0) for b in test_ds], axis=0)
    pred_idx    = np.argmax(all_probs, axis=1)
    pred_labels = [class_names[i] for i in pred_idx]
    confs       = [round(float(all_probs[i, pred_idx[i]]), 4) for i in range(len(pred_idx))]

    result_df = pd.DataFrame({"filename": all_files, "predicted_label": pred_labels, "confidence": confs})
    out_csv   = os.path.join(test_dir, "prediction_results.csv")
    result_df.to_csv(out_csv, index=False)
    print(f"[INFO] ✅ Hasil disimpan: {out_csv}")

    pd.Series(pred_labels).value_counts().sort_index().plot(
        kind='bar', color='#3498db', edgecolor='white', figsize=(10, 4))
    plt.title('Distribusi Prediksi'); plt.xlabel('Kelas'); plt.ylabel('Jumlah')
    plt.xticks(rotation=30, ha='right'); plt.grid(axis='y', alpha=0.3)
    plt.tight_layout(); plt.show()
    print(result_df.to_string(index=False))
    return result_df

result_df = predict_all_in_folder(inf_model, class_names_inf, CUSTOM_TEST_DIR)
if result_df is not None: result_df.head(20)

---
## 🍃 14) Leaf Segmentation + Inference — GrabCut Auto-Seed

**Mengapa GrabCut lebih baik dari HSV biasa?**

| | HSV Basic | GrabCut Auto-Seed |
|---|:---:|:---:|
| Daun sakit (pucat/kuning) | ❌ Tidak terdeteksi | ✅ Terdeteksi |
| Background hijau kompleks | ❌ Ikut terdeteksi | ✅ Terbuang |
| Domain shift training↔inference | ❌ Ada | ✅ Tidak ada |
| Kecepatan | ⚡⚡⚡ | ⚡ |

**Strategi GrabCut Auto-Seed (3 Zona):**

```
┌─────────────────────────────────────────────────┐
│  Sure FG  → H=[25-95], S≥60, V≥40              │
│             Pasti daun (hijau jelas)             │
│  Sure BG  → V≤20 | V≥245 | S≤10               │
│             Pasti background (gelap/abu/putih)   │
│  Unknown  → Sisa piksel                         │
│             GrabCut putuskan via GMM (otomatis)  │
└─────────────────────────────────────────────────┘
Fallback: jika Sure FG < 0.5% gambar → rect-based GrabCut
```

**Pipeline:**
```
Gambar asli
  → grabcut_leaf_mask_autoseed()  ← 3-zona seed
  → refine_mask()                 ← CLOSE+OPEN morph
  → keep_largest_component()      ← blob daun terbesar
  → bitwise_and()                 ← BG = HITAM ⬛
  → crop_by_mask()               ← crop ketat ke daun
  → resize(224,224) → RGB         ← input model
```

| Cell | Fungsi |
|:---:|---|
| **14b** | Prediksi 1 gambar — **4 panel** (Asli \| Mask \| BG Hitam \| Input Model) |
| **14c** | Batch prediksi semua gambar → `prediction_results_grabcut.csv` |

> ⚠️ **Prasyarat:** `Cell 2b` (GrabCut functions) + `Cell 13a` (load model) harus dijalankan dulu.


In [ ]:
# ----------------------------
# 14b) Prediksi 1 Gambar — GrabCut Auto-Seed
# ----------------------------
# ⚙️ UBAH DI SINI
IMAGE_FILE       = "image.png"   # nama file di test_images/
USE_SEGMENTATION = True          # True = GrabCut | False = tanpa segmentasi
# ⚙️

IMAGE_PATH = os.path.join(CUSTOM_TEST_DIR, IMAGE_FILE)
if not os.path.exists(IMAGE_PATH):
    raise FileNotFoundError(f"❌ File tidak ada: {IMAGE_PATH}")

print(f"File : {IMAGE_PATH}")
print(f"Mode : {'GrabCut Auto-Seed' if USE_SEGMENTATION else 'Tanpa Segmentasi'}")

if USE_SEGMENTATION:
    bgr_in = cv2.imread(IMAGE_PATH)
    rgb_input, leaf_mask, leaf_only, bbox = preprocess_leaf_grabcut(bgr_in, IMG_SIZE)

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle("GrabCut Auto-Seed — Hanya Daun, Background Hitam ⬛",
                 fontsize=13, fontweight="bold")
    axes[0].imshow(cv2.cvtColor(bgr_in, cv2.COLOR_BGR2RGB))
    axes[0].set_title("1️⃣  Gambar Asli");            axes[0].axis("off")
    axes[1].imshow(leaf_mask, cmap="Greens")
    axes[1].set_title("2️⃣  Mask Daun (GrabCut)");    axes[1].axis("off")
    axes[2].imshow(cv2.cvtColor(leaf_only, cv2.COLOR_BGR2RGB))
    axes[2].set_title("3️⃣  Daun + BG Hitam ⬛");     axes[2].axis("off")
    axes[3].imshow(rgb_input.astype("uint8"))
    axes[3].set_title("4️⃣  Input Model 224×224");    axes[3].axis("off")
    plt.tight_layout(); plt.show()
    img_tensor = tf.expand_dims(tf.constant(rgb_input), axis=0)
else:
    raw = tf.io.read_file(IMAGE_PATH)
    img = tf.image.decode_image(raw, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE, method="bilinear")
    img_tensor = tf.expand_dims(tf.cast(img, tf.float32), axis=0)

probs      = inf_model.predict(img_tensor, verbose=0)[0]
pred_idx   = int(np.argmax(probs))
pred_label = class_names_inf[pred_idx]
confidence = float(probs[pred_idx])

fig2, ax2 = plt.subplots(figsize=(9, 4))
colors = ["#27ae60" if i==pred_idx else "#3498db" for i in range(len(class_names_inf))]
bars   = ax2.barh(class_names_inf, probs*100, color=colors, edgecolor="white", height=0.6)
clr    = "#27ae60" if confidence>=0.8 else ("#e67e22" if confidence>=0.5 else "#e74c3c")
ax2.set_title(f"🏷️  {pred_label.replace('_',' ').title()}  |  {confidence*100:.2f}%",
              fontsize=12, fontweight="bold", color=clr)
ax2.set_xlabel("Probability (%)"); ax2.set_xlim(0, 105)
for bar, val in zip(bars, probs):
    ax2.text(val*100+0.8, bar.get_y()+bar.get_height()/2,
             f"{val*100:.1f}%", va="center", fontsize=9)
ax2.legend(handles=[mpatches.Patch(color="#27ae60", label="Predicted"),
                    mpatches.Patch(color="#3498db", label="Others")], loc="lower right")
ax2.grid(axis="x", alpha=0.3); plt.tight_layout(); plt.show()
print(f"\n🎯 Label     : {pred_label}")
print(f"   Confidence: {confidence*100:.2f}%")


### 14c. Batch Prediksi Semua Gambar → CSV (GrabCut Auto-Seed)

Proses semua gambar di `test_images/` dengan GrabCut segmentation.
Hasil disimpan ke `test_images/prediction_results_grabcut.csv`.


In [ ]:
# ----------------------------
# 14c) Batch Prediction DENGAN GrabCut → CSV
# ----------------------------
# ⚙️ UBAH DI SINI
USE_SEGMENTATION = True
# ⚙️

valid_ext = (".jpg",".jpeg",".png",".JPG",".JPEG",".PNG")
all_files = sorted([f for f in os.listdir(CUSTOM_TEST_DIR) if f.endswith(valid_ext)])

if not all_files:
    print(f"[WARN] Tidak ada gambar di '{CUSTOM_TEST_DIR}'.")
else:
    mode = "GrabCut" if USE_SEGMENTATION else "RAW"
    print(f"[INFO] {len(all_files)} gambar | Mode: {mode}")
    results = []

    for fname in all_files:
        fpath = os.path.join(CUSTOM_TEST_DIR, fname)
        try:
            if USE_SEGMENTATION:
                bgr_f = cv2.imread(fpath)
                rgb_seg, _, _, _ = preprocess_leaf_grabcut(bgr_f, IMG_SIZE)
                t = tf.expand_dims(tf.constant(rgb_seg), axis=0)
            else:
                raw = tf.io.read_file(fpath)
                img = tf.image.decode_image(raw, channels=3, expand_animations=False)
                img = tf.image.resize(img, IMG_SIZE, method="bilinear")
                t   = tf.expand_dims(tf.cast(img, tf.float32), axis=0)
            probs = inf_model.predict(t, verbose=0)[0]
            pi    = int(np.argmax(probs))
            lbl   = class_names_inf[pi]
            conf  = round(float(probs[pi]), 4)
            st    = "OK"
        except Exception as e:
            lbl="ERROR"; conf=0.0; st=str(e)
        results.append({"filename":fname,"predicted_label":lbl,"confidence":conf,"status":st})
        print(f"  [{st}] {fname:30s} → {lbl:20s} ({conf*100:.1f}%)")

    suffix     = "grabcut" if USE_SEGMENTATION else "raw"
    result_df  = pd.DataFrame(results)
    out_csv    = os.path.join(CUSTOM_TEST_DIR, f"prediction_results_{suffix}.csv")
    result_df.to_csv(out_csv, index=False)
    print(f"\n✅ Hasil: {out_csv}")

    ok = result_df[result_df.status=="OK"]
    if not ok.empty:
        ok["predicted_label"].value_counts().sort_index().plot(
            kind="bar", color="#27ae60", edgecolor="white", figsize=(10,4))
        plt.title(f"Distribusi Prediksi ({mode})")
        plt.xlabel("Kelas"); plt.ylabel("Jumlah")
        plt.xticks(rotation=30, ha="right"); plt.grid(axis="y", alpha=0.3)
        plt.tight_layout(); plt.show()
    print(result_df.to_string(index=False))
